# Init

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import kaiser

In [ ]:
%matplotlib qt5

In [ ]:
import sys

In [ ]:
sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia
import toolsdiadem.ser as ser

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cmap='jet'
withPlots = 0
center = 6650
width = 500

In [ ]:
base_path = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15"
dic_dir = {"190130" : ["absC"],
           "190131" : ["absAHP12", "absB", "absD", "absJ"],
           "190201" : ["absA", "absAHP20", "absE", "absG", "absM"],
           "190204" : ["absAHP9", "absAHP30", "absAHP45", "absH", "absI", "absK", "absL", "absN", "absO", "absU"]}

In [ ]:
dir_A     = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190201/absA"
dir_C     = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190128/absC"
dir_AHP9  = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190204/absAHP9"
dir_AHP45 = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190204/absAHP45"

# Fond de chambre

In [ ]:
tmp = np.loadtxt(f"{base_path}/S11_fond_de_chambre.txt", skiprows=1)
S11_fondDeChambre = tmp[:,1] + 1j * tmp[:,2]
tmp = np.loadtxt(f"{base_path}/S11_fond_de_chambre_withoutAHP45.txt", skiprows=1)
S11_fondDeChambre_withoutAHP45 = tmp[:,1] + 1j * tmp[:,2]

In [ ]:
fig, ax = plt.subplots(1,1)
plt.plot( 20 * np.log10( np.abs( np.fft.ifft( S11_fondDeChambre ) ) ),
         label="fond de chambre" )
plt.plot( 20 * np.log10( np.abs( np.fft.ifft( S11_fondDeChambre_withoutAHP45 ) ) ),
         label="fond de chambre without AHP45" )
plt.grid()
plt.legend()

# Plaque de référence

In [ ]:
dir_plaqueDeReference = f"{base_path}/190129/caracterisation/plaque_ref"
freq_all, S11_plaque_ref = ser.getFreqAndS11( dir_plaqueDeReference )
S11_plaque_ref_sans_fondDeChambre = S11_plaque_ref - S11_fondDeChambre
d_plaqueDeReference = np.linspace( 0, 300 / ( freq_all[1] - freq_all[0] ), freq_all.size ) * 0.5

In [ ]:
idxElevation = 20
fig, ax = plt.subplots(1,1)
ax.plot( d_plaqueDeReference, 20 * np.log10( np.abs( np.fft.ifft( S11_plaque_ref ) ) ),
         label="S11, plaque_ref, time domain" )
ax.plot( d_plaqueDeReference, 20 * np.log10( np.abs( np.fft.ifft( S11_plaque_ref_sans_fondDeChambre ) ) ),
         label="S11, plaque_ref moins fond de chambre, time domain" )
ax.grid()
ax.legend()
ax.set_xlabel("distance[mm]")

# Réflectivité
ref: ser4siepel_multiband_scan_elev_Reflectivite.sce

## Initialisation des variables

In [ ]:
nb_elev = 41
nb_freq = 2001

elevation        = np.linspace( 0.76 - 1.0, 0.76 + 1.0, nb_elev )
data_S11         = np.zeros( (nb_elev, nb_freq), dtype=complex )
data_S11_td      = np.zeros( (nb_elev, nb_freq), dtype=complex )
data_S11_tg      = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy    = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy_td = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy_tg = np.zeros( (nb_elev, nb_freq), dtype=complex )
reflectivite     = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_moy          = np.zeros( nb_freq,            dtype=complex )
S11_moy_square   = np.zeros( nb_freq,            dtype=complex )

## Lecture des données et time gating

### Lecture des données

In [ ]:
seq = 0
flag_gate = 1
data_dir = dir_AHP45

for ielev in range(nb_elev):
    freq_all, S11_all = ser.getFreqAndS11( data_dir, seq0=seq, verbose=0 )
    seq = seq + 3
    data_S11[ ielev, : ] = S11_all
    
d = np.linspace( 0, 300 / ( freq_all[1] - freq_all[0] ), nb_freq ) * 0.5

### Gating

In [ ]:
gate = ser.getWindow( d, center, width, id="llc" )

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(d, gate, label='gate')
ax.legend()
ax.grid()
fig.savefig("gate.png")

In [ ]:
S11_moy     = np.average( data_S11, axis=0 )
S11_moy_td  = np.fft.ifft( S11_moy )
S11_moy_tg  = S11_moy_td * gate

data_S11_td = np.fft.ifft( data_S11, axis=1 )
data_S11_tg = data_S11_td * gate

In [ ]:
idxElevation = 20
fig, ax = plt.subplots(1,1)
ax.plot( d, 20*np.log10( np.abs( data_S11_td[idxElevation,:] ) ), label="S11, time domain" )
ax.plot( d, 20*np.log10( np.abs( data_S11_tg[idxElevation,:] ) ), label="S11, time domain, gated" )
ax.legend()
ax.grid()
ax.set_xlabel( "distance [mm]" )
ax.set_title( f"elevation {elevation[idxElevation]}°" )

In [ ]:
idxElevation = 20
fig, ax = plt.subplots(1,1)
ax.plot( freq_all, 20*np.log10( np.abs( np.fft.fft( data_S11_td[idxElevation,:] ) ) ), label="S11" )
ax.plot( freq_all, 20*np.log10( np.abs( np.fft.fft( data_S11_tg[idxElevation,:] ) ) ), label="S11, time gated" )
ax.legend()
ax.grid()
ax.set_xlabel( "frequence [GHz]" )
ax.set_title( f"elevation {elevation[idxElevation]}°" )

## Calcul de la réflectivité

In [ ]:
def getReflectivite( S11, plaqueRef, fondDeChambre, gate ):
    S11_plaque_ref_tg = np.fft.fft( np.fft.ifft( plaqueRef - fondDeChambre ) * gate )
    S11_minus_fdc    = S11 - fondDeChambre
    s11_minus_fdc_td = np.fft.ifft( S11_minus_fdc, axis=1 )
    s11_minus_fdc_tg = s11_minus_moy_td * gate
    S11_minus_moy_tg = np.fft.fft( s11_minus_fdc_tg, axis=1 )
    reflectivite     = S11_minus_moy_tg / S11_plaque_ref_tg
    return reflectivite

### Avec fond de chambre complet

In [ ]:
S11_plaque_ref_tg = np.fft.fft( np.fft.ifft( S11_plaque_ref - S11_fondDeChambre ) * gate )
reflectivite = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy    = data_S11 - S11_fondDeChambre
s11_minus_moy_td = np.fft.ifft( S11_minus_moy, axis=1 )
s11_minus_moy_tg = s11_minus_moy_td * gate
S11_minus_moy_tg = np.fft.fft( s11_minus_moy_tg, axis=1 )
reflectivite     = S11_minus_moy_tg / S11_plaque_ref_tg

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot( d, 20*np.log10( np.abs( data_S11_td[20,:] ) ), label="S11, time domain" )
ax.plot( d, 20*np.log10( np.abs( data_S11_tg[20,:] ) ), label="S11, time domain, gated" )
ax.plot( d, 20*np.log10( np.abs( s11_minus_moy_td[20,:] ) ), label="S11 moins fond de chambre, time domain" )
ax.plot( d, 20*np.log10( np.abs( s11_minus_moy_tg[20,:] ) ), label="S11 moins fond de chambre, time domain, gated" )
ax.legend()
ax.grid()

### Calcul de la réflectivité avec fond de chambre sans AHP45

In [ ]:
reflectivite_a = ser.getReflectivite( data_S11, S11_plaque_ref, S11_fondDeChambre, gate )
reflectivite_b = ser.getReflectivite( data_S11, S11_plaque_ref, S11_fondDeChambre_withoutAHP45, gate )

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot( d, 20*np.log10( np.abs( reflectivite_a[20,:] ) ), label="reflectivite_a" )
ax.plot( d, 20*np.log10( np.abs( reflectivite_b[20,:] ) ), label="reflectivite_a" )
ax.legend()
ax.grid()

# Plot

## Coefficient de réflexion

In [ ]:
cmap='jet'

toPlot = 20 * np.log10( np.abs( ( S11_minus_moy_tg ) ) )

valMin = -140
toPlot = 0.5 * ( np.abs( toPlot - valMin ) + toPlot - valMin ) + valMin
valMax = np.amax( toPlot )
valMin = np.amin( toPlot )
print( f"valMax = {valMax:.2f} valMin = {valMin:.2f}")

fig, ax = plt.subplots( 1, 1 )
im = plt.pcolormesh( freq_all, elevation, toPlot, cmap='jet' )
dia.addColorBar(  im, ax, aspect='auto' )
title = f"Coefficient de réflexion de l'absorbant\n{data_dir}"
ax.set_title(title)
ax.set_ylabel( "Elévation [°]" )
ax.set_xlabel( "Fréquence [GHz]" )

## Réflectivité de l'absorbant

In [ ]:
toPlot = np.zeros( (len(elevation), len(tab_freq)) )
gatebis  = 0.0 * gate
gatebis[ np.where( np.abs( freq_all - 62.5 ) < 12.4 ) ] = 1.0
toPlot = 20 * np.log10( 1e-12 + np.abs( reflectivite * gatebis ) )

valMin = -90
toPlot = 0.5 * ( np.abs( toPlot - valMin ) + toPlot - valMin ) + valMin
valMax = np.amax( toPlot )
valMin = np.amin( toPlot )
print( f"valMax = {valMax:.2f} valMin = {valMin:.2f}")
     
vmin = -80
vmax = -0
fig, ax = plt.subplots( 1, 1 )
im = plt.pcolormesh( elevation, np.flip( freq_all ), np.rot90(toPlot), cmap=cmap )

title = f"Réflectivité absorbant\n{data_dir}"
ax.set_title(title)
ax.set_xlabel( "Elévation [°]" )
ax.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax, aspect='auto' )